# First Year Project

## Project Description

<hr>

### Imports and Constants

In [ ]:
# Std
from pathlib import Path
import csv
import json
import platform
from IPython.display import display as d

# Array manipulation
import numpy as np
import pandas as pd


# Statistics
from scipy.stats import chi2_contingency

# Plots
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Visual
import shapely
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString, Polygon, MultiPolygon

import folium
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster

%run -i ../scripts/functions.py

In [ ]:
# Logging levels
LOG = True

# Data paths
PATH = {
    'raw': Path('../data/raw/'),
    'processed': Path('../data/processed/'),
    'interim': Path('../data/interim/'),
    'references': Path('../references')
}

# Raw paths
PATH['raw_accident'] = PATH['raw'] / "Road Safety Data - Accidents 2019.csv"
PATH['raw_casual'] = PATH['raw'] / "Road Safety Data - Casualties 2019.csv"
PATH['raw_vehicles'] = PATH['raw'] / "Road Safety Data- Vehicles 2019.csv"

# Interim paths
PATH['accident'] = PATH['interim'] / "bradford_accidents.csv"
PATH['casual'] = PATH['interim'] / "bradford_casualties.csv"
PATH['vehicles'] = PATH['interim'] / "bradford_vehicles.csv"

# Ref paths
PATH['var_lookup'] = PATH['references'] / 'variable lookup.xls'


EXCEL_LABELS = pd.read_excel(PATH['var_lookup'], sheet_name = None, index_col = 0)
VAR = {}

# maps special sheet name to full name for later proccesing
special_sheet_names = {'Ped_Location': 'Pedestrian_Location', \
                  'Ped_Movement': 'Pedestrian_Movement',
                      'Speed_Limit': 'Speed_limit'}

for sheet_name in EXCEL_LABELS:
    # add underscores in field name since its missing in excel
    fixed_sheet_name = sheet_name.replace(" ", "_")
    
    # handle sheet names that are significally different named compared to the table headers
    if fixed_sheet_name in special_sheet_names:
        fixed_sheet_name = special_sheet_names[fixed_sheet_name]
    
    # create new dict with renamed keys
    VAR[fixed_sheet_name] = EXCEL_LABELS[sheet_name]


Special function for labels

In [ ]:
def get_unique_label_fld_lookup(data=None, colname=None):
    '''Returns a DataFrame with only the labels needed to set xticklabels
    When settting xticklabels, we often have more labels in VAR than present in our dataset.
    in a way, this function creates a copy of VAR with only the labels used in our data (bradford)
    
    :param dataset: A Pandas DataFrame, eg DATA['accident']
    :param column: Column name as string, with underscores
    '''
    # problem: not all labels are used in 'Special Conditions at Site'
    # solution: remove the labelse not present from VAR['Special Conditions at Site']
    # used_label_codes = pd.unique(DATA['accident']['Special_Conditions_at_Site'])
    used_label_codes = pd.unique(data[colname])

    # Save all labels for this column.
    all_labels = VAR[colname]

    # set default values of the relevant labels we want to keep
    unique_label_fld_lookup = all_labels

    # loop through all label codes (they are set as the index)
    for label_code in all_labels.index:
        if label_code not in used_label_codes:
            # if the label is not used 
            unique_label_fld_lookup.drop(index=label_code, inplace=True)

    return unique_label_fld_lookup

<hr>

## Task 0 - Data filtering and cleaning

Most of our data filtering and cleaning happends from scripts we wrote,
which takes in the raw files as input and process them to only show bradford.

They can be run from the terminal or here. It's in src/scripts.

Pandas can handle and process datasets even though they're missing.
We're categorizing missing values as N/A.

In [ ]:
# Process raw files into bradford files with pandas(pd)
# Depends on paths from this notebook, so run for here or alter to be independant.
%run -i ../src/scripts/process_bradford_in.py

In [ ]:
DATA = {}

# Read data and explicitly define some data types for columns.
# The datasets are saved in utf8 with bom, so we need utf-8-sig encoding.
# We treat explicit -1 from the datasets as NA because it's their definition from the documentation.

DATA['accident'] = pd.read_csv(PATH['accident'], dtype={0: 'string', 31: 'string'}, encoding='utf-8-sig', na_values="-1")
DATA['casual'] = pd.read_csv(PATH['casual'], dtype={0: 'string'}, encoding='utf-8-sig', na_values="-1")
DATA['vehicles'] = pd.read_csv(PATH['vehicles'], dtype={0: 'string'}, encoding='utf-8-sig', na_values="-1")

<hr>

In [ ]:
# Missing values overview

# Quick overview
DATA['accident'].isna().any()

# Get only the names of the columns with missing values: Time, Junction_Control and 2nd_Road_Class
DATA['accident'].columns[DATA['accident'].isna().any()]

# Pandas: first sum the missing values row-wise, then sum all the row-wise sums to one single sum
# Missing                            Not missing
DATA['accident'].isna().sum().sum(), DATA['accident'].notna().sum().sum()

# Return the rows with missing values
# We use d() to display multiple elements
d(DATA['accident'][DATA['accident']['Time'].isna()])
d(DATA['accident'][DATA['accident']['Junction_Control'].isna()]['Junction_Control'])
d(DATA['accident'][DATA['accident']['2nd_Road_Class'].isna()]['Junction_Control'])

## Task 1 - Single variable analysis

In [ ]:
# Age analysis
pd.concat([VAR['Age_Band'], DATA['casual']['Age_Band_of_Casualty'].value_counts()], axis=1)

In [ ]:
Age=VAR['Age_Band']['label'][:11]
Numbers=DATA['casual']['Age_Band_of_Casualty'].value_counts(sort=False)


fig = plt.figure(figsize=(7, 3))
axes = fig.add_axes([0, 0, 1, 1])
plt.bar(Age, Numbers,width=0.5, color='blue', edgecolor='black')
axes.set_ylabel('Number of Casualties');axes.set_xlabel('Age Range');axes.set_title('Number of Casualties regarding their ages');

In [ ]:
# Time analysis
pd.concat([VAR['Day_of_Week'], DATA['accident']['Day_of_Week'].value_counts()], axis=1)

In [ ]:
Weekdays=VAR['Day_of_Week']['label']
Numbers=DATA['accident']['Day_of_Week'].value_counts(sort=False)
    
    
fig = plt.figure(figsize=(6, 3))
axes = fig.add_axes([0, 0, 1, 1])
plt.bar(Weekdays, Numbers,width=0.5, color='blue', edgecolor='black')
axes.set_ylabel('Number of Accidents');axes.set_xlabel('Days of the Week');axes.set_title('Total number of Accidents regarding days of the week');

In [ ]:
# Movement analysis - Task 1, c
pd.concat([VAR['Pedestrian_Movement'], DATA['casual']['Pedestrian_Movement'].value_counts()], axis=1)

In [ ]:
PedMove=VAR['Pedestrian_Movement']['label'][:10]
Numbers=DATA['casual']['Pedestrian_Movement'].value_counts(sort=False)

fig = plt.figure(figsize=(6, 3))
axes = fig.add_axes([0, 0, 1, 1])
plt.barh(PedMove, Numbers, color='blue', edgecolor='black')
axes.set_xlabel('Number of Casualties');axes.set_title('Number of Casualties regarding the movement of the Pedestrian');

<hr>

## Task 2: Associations

We want to see if there's an association between the casualty severity from the casualties table, and a field from the accident table. We focus on casualty serverity associatied with a road type or road charecteristic.  

Casualty serverity vs
- Road Type
- Junction Detail
- Speed limit

We choose these since they are deemed best suited to shed light on our research question. The dataset are full of categorical varibles, including the ones above, so we choose to test for statistical assocaition using Pearson $\chi^2$ test of independence. See references for more information on this test

Every single casualty was part of an accident. So in order to get fields from diffrent tables we do an inner join to get the extra information about the casualty. 

In [ ]:
merged = DATA['casual'].merge(DATA['accident'], how="inner")
merged

### Association test between Casualty_Severity and Road_Type

Make contingency table.

In [ ]:
observed_pd1 = pd.crosstab(merged["Road_Type"], merged["Casualty_Severity"])
observed1 = observed_pd1.to_numpy()
observed_pd1

We remove the road types with almost no accidents since we are not intersted in them, if there's no casualties happening.  
The Pearson chi-square test should only be used if most cells have an expected count above 5, and the minimum expected count is at least 1.  
Since we have 3 columns we say that we want the rows to sum to least $5*3 = 15$, so we obmit some rows using a mask.

In [ ]:
# create mask
observed_pd1_mask = observed_pd1.sum(axis=1) >= 15
# use mask to change data
observed_pd1_filtered = observed_pd1[observed_pd1_mask]
observed1 = observed_pd1_filtered.to_numpy()
observed_pd1_filtered

We compare these observed values with expected values. This will tell there's an association between the variables.
We calculate cramers V to see how strong the assocation is.

In [ ]:
chiVal1, pVal1, df1, observed1, expected1, V1 = get_pearson_test(observed=observed1)
chiVal1, pVal1, df1, observed1, expected1.astype(int), V1

We create labels and remove those not needed.

In [ ]:
var1_labels1 = dict(get_unique_label_fld_lookup(data=merged, colname="Road_Type").label)
var2_labels1 = list(VAR["Casualty_Severity"].label)
var1_labels1

In [ ]:
labelcodestoremove1 = []
for labelcode in var1_labels1:
    if labelcode not in observed_pd1_filtered.index:
        labelcodestoremove1.append(labelcode)
        
for label in labelcodestoremove1:
    var1_labels1.pop(label)
var1_labels1

In [ ]:
title1 = "Pearson test of independence: Casualty severity vs Road Type"

In [ ]:
plot_pearson_test(observed=observed1, expected=expected1, var1_labels=var1_labels1, var2_labels=var2_labels1, title=title1)

### Association test between Casualty_Severity vs Junction_Detail

The same proceduce is done below for the two other fields.

In [ ]:
observed_pd2 = pd.crosstab(merged["Junction_Detail"], merged["Casualty_Severity"], margins=False)
observed2 = observed_pd2.to_numpy()

In [ ]:
# create mask
observed_pd2_mask = observed_pd2.sum(axis=1) >= 15
# use make to change data
observed_pd2_filtered = observed_pd2[observed_pd2_mask]
observed2 = observed_pd2_filtered.to_numpy()

In [ ]:
var1_labels2 = dict(get_unique_label_fld_lookup(data=merged, colname="Junction_Detail").label)
var2_labels2 = list(VAR["Casualty_Severity"].label)

In [ ]:
labelcodestoremove = []
for labelcode in var1_labels2:
    if labelcode not in observed_pd2_filtered.index:
        labelcodestoremove.append(labelcode)
        
for label in labelcodestoremove:
    var1_labels2.pop(label)

In [ ]:
chiVal2, pVal2, df2, observed2, expected2, V2 = get_pearson_test(observed=observed2)
chiVal2, pVal2, df2, observed2, expected2.astype(int), V2

In [ ]:
title2 = "Pearson test of independence: Casualty severity vs Junction detail "

In [ ]:
plot_pearson_test(observed=observed2, expected=expected2, var1_labels=var1_labels2, var2_labels=var2_labels2, title=title2)

### Association test between Casualty_Severity vs Speed_limit

In [ ]:
observed_pd3 = pd.crosstab(merged["Speed_limit"], merged["Casualty_Severity"], margins=False)
observed3 = observed_pd3.to_numpy()

In [ ]:
# create mask
observed_pd3_mask = observed_pd3.sum(axis=1) >= 15
# use make to change data
observed_pd3_filtered = observed_pd3[observed_pd3_mask]
observed3 = observed_pd3_filtered.to_numpy()

In [ ]:
var1_labels3 = dict(get_unique_label_fld_lookup(data=merged, colname="Speed_limit").label)
var2_labels3 = list(VAR["Casualty_Severity"].label)

In [ ]:
labelcodestoremove3 = []
for labelcode in var1_labels3:
    if labelcode not in observed_pd3_filtered.index:
        labelcodestoremove3.append(labelcode)
        
for label in labelcodestoremove3:
    var1_labels3.pop(label)

In [ ]:
chiVal3, pVal3, df3, observed3, expected3, V3 = get_pearson_test(observed=observed3)
chiVal3, pVal3, df3, observed3, expected3.astype(int), V3

In [ ]:
title3 = "Pearson test of independence: Casualty severity vs Speed Limit"

In [ ]:
plot_pearson_test(observed=observed3, expected=expected3, var1_labels=var1_labels3, var2_labels=var2_labels3, title=title3)

<hr>

## Task 3: Map visualization

In [ ]:
with open(PATH["processed"] / "citieslad.json") as f:
    cities_json = json.load(f)

# cities_json['Bradford']

In [ ]:
latlons = pd.concat([DATA["accident"]['Latitude'], DATA["accident"]['Longitude']], axis=1).values.tolist()

centroid = list(MultiPoint(latlons).centroid.coords)[0]

m1 = folium.Map(centroid, zoom_start=11)

for i, row in DATA["accident"].iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
        radius = 5,
        popup = row['Accident_Index'] + "\n" + row["Date"] + ", " + str(row["Time"]),
        fill_color = "#3db7e4",
    ).add_to(m1)

HeatMap(latlons).add_to(folium.FeatureGroup(name='Heat Map').add_to(m1))
folium.LayerControl().add_to(m1)
m1

In [ ]:
# latlons = pd.concat([DATA["accident"]["Latitude"], DATA['accident']['Longitude']], axis = 1).values.tolist()

# centroid = list(MultiPoint(latlons).centroid.coords)[0]

m2 = folium.Map(centroid, zoom_start = 11)

marker_cluster = MarkerCluster().add_to(folium.FeatureGroup(name='Clusters').add_to(m2))

for i, row in DATA['accident'].iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
        radius = 5, 
        popup = row['Accident_Index'] + '\n' + row['Date'] + ", " + str(row['Time']),
        fill_color = "#3db7e4",
    ).add_to(marker_cluster)
    
    HeatMap(latlons).add_to(folium.FeatureGroup(name='Heat Map').add_to(m2))
    folium.LayerControl().add_to(m2)
m2

<hr>

## Task 4: Open question

<hr>